In [6]:
# args

# persona_val_path = "opinions_qa/persona_val/American_Trends_Panel_W34/date0828_personas_American_Trends_Panel_W34_testonly_haiku.json"
persona_val_path = "opinions_qa/persona_val/American_Trends_Panel_W34/date0831_personas_full_haiku_known_test.json"


In [7]:
import os
os.chdir("/home/ubuntu/code/pump_post_midterm/pump")

from transformers import AutoTokenizer, AutoModel
import json



In [8]:
with open(persona_val_path, 'r') as f:
    p_vals = json.load(f)
print(len(list(p_vals.keys())))

661


In [9]:
import random
import pandas as pd
from src.utils import get_file_from_s3
random.seed(42)

file_key = "American_Trends_Panel_W34"
resp_df = pd.read_csv(get_file_from_s3(f"human_resp/{file_key}/responses.csv"))
test_user_idx = random.choices(range(len(resp_df)), k=int(len(resp_df)*0.1))
test_resp_df = resp_df.iloc[test_user_idx]
test_len = int(len(resp_df)*0.1)

test_user_list = list(p_vals.keys())[:test_len]
train_user_list = list(p_vals.keys())[test_len:]

print('test:', len(test_user_list))
print('train:', len(train_user_list))

test: 253
train: 408


In [10]:
def get_user_pval_df(pvals, user_list):
    records = []
    for user in user_list:
        personas = pvals[user]
        entry = {'user': user}
        for p in personas:
            entry[p['name']] = p['inferred_value']
        records.append(entry)

    df = pd.DataFrame(records)
    return df

In [11]:
df = get_user_pval_df(p_vals, train_user_list)
df[:3]

,user,Safety vs. Timeliness Prioritization,Health and Nutrition Values,Personal Values on Gene Editing,Concern for Social Justice,Worldview and Belief System,Openness to Food Innovation,Trust in Science and Technology,Lifestyle Priorities,Core Personal Values,...,Attitude Towards Science,Risk Perception,Technological Outlook,Personal Health Concerns,Technological Awareness,Dietary Habits,Healthcare System Familiarity,Information Sources on GMOs,Access to Healthcare,Personal Experiences with Scientific and Technological Advancements
0,13,prioritizes safety over speed,balanced values,values caution and preservation,low concern,conservative,resistant to innovation,low trust,prioritizes other lifestyle factors,prioritizes individual freedom,...,negative,low risk perception,pessimistic,does not have personal health concerns,low awareness,consistently unhealthy,low familiarity,non-scientific sources,low access,negative experiences
1,14,prioritizes safety over speed,balanced values,values caution and preservation,moderate concern,moderate,resistant to innovation,low trust,balanced priorities,prioritizes safety,...,negative,low risk perception,pessimistic,does not have personal health concerns,low awareness,balanced,moderate familiarity,trusts non-scientific sources,low access,negative experiences
2,15,prioritizes safety over speed,balanced values,values caution and preservation,high concern,moderate,open to innovation,high trust,prioritizes health and nutrition,prioritizes individual freedom,...,positive,low risk perception,optimistic,has personal health concerns,low awareness,balanced,moderate familiarity,trusts scientific sources,low access,positive experiences


In [12]:
from collections import Counter

skew_cnt = 0
for col in df.columns:
    if col == 'user': continue

    cnt = Counter(df[col].values)
    if len(cnt.keys()) < 2: 
        skew_cnt += 1 
        print(col)
        print(Counter(df[col].values))
        print()
        continue
    first, second = cnt.most_common(2)
    if first[1] / second[1] > 10:
        skew_cnt += 1 
        print(col)
        print(Counter(df[col].values))
        print()
print(skew_cnt / len(df.columns))

Safety vs. Timeliness Prioritization
Counter({'prioritizes safety over speed': 402, 'prioritizes speed over safety': 6})

Health and Nutrition Values
Counter({'balanced values': 397, 'values health and wellness': 6, 'values convenience and indulgence': 5})

Personal Values on Gene Editing
Counter({'values caution and preservation': 406, 'values progress and innovation': 2})

Personal Values Towards Genetically Modified Foods
Counter({'prioritizes health and safety': 392, 'prioritizes convenience and cost': 16})

Ethical Orientation
Counter({'low ethical concern': 407, 'high ethical concern': 1})

Attitudes Towards Genetic Engineering
Counter({'skeptical': 407, 'optimistic': 1})

Self-Discipline
Counter({'moderate': 387, 'low': 21})

Perception of Technological Equity
Counter({'indifferent to inequality': 385, 'concerned about inequality': 23})

Food Motivations
Counter({'balanced': 408})

Nutrition Knowledge
Counter({'moderate': 399, 'low': 8, 'high': 1})

Perception of Gene Editing Im

In [13]:
su = test_user_list[0]

def get_user_persona_repr(personas):
    return '; '.join(f"{p['inferred_value']}" for p in personas)

get_user_persona_repr(p_vals[su])

'prioritizes safety over speed; balanced values; values caution and preservation; moderate concern; moderate; resistant to innovation; low trust; balanced priorities; prioritizes individual freedom; high ethical concerns; prioritizes health and safety; prioritizes individual autonomy and freedom; high trust; skeptical; low environmental consciousness; low risk perception; moderate; indifferent to inequality; does not see it as a problem; balanced; moderate health consciousness; neutral; indifferent to food type; low ethical concern; moderate; moderate; negative impact; less concerned about speed of innovation; moderate; negative; low risk perception; neutral; has personal health concerns; low awareness; balanced; low familiarity; trusts non-scientific sources; low access; negative experiences'

In [14]:


import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import time

# Each query needs to be accompanied by an corresponding instruction describing the task.
task_name_to_instruct = {"retrieve_user_personas": "Given a user persona description, retrieve similar user presona descriptions",}

query_prefix = "Instruct: "+task_name_to_instruct["retrieve_user_personas"]+"\nQuery: "
queries = [
    get_user_persona_repr(p_vals[user]) for user in test_user_list
]

# No instruction needed for retrieval passages
passage_prefix = ""
passages = [
    get_user_persona_repr(p_vals[user]) for user in train_user_list[:200]
]

# load model with tokenizer
model = AutoModel.from_pretrained('nvidia/NV-Embed-v2', trust_remote_code=True, device_map='auto')

# get the embeddings
max_length = 4096
tic = time.time()
# query_embeddings = model.encode(queries, instruction=query_prefix, max_length=max_length)
# passage_embeddings = model.encode(passages, instruction=passage_prefix, max_length=max_length)
batch_size=2
query_embeddings = model._do_encode(queries, batch_size=batch_size, instruction=query_prefix, max_length=max_length, num_workers=32, return_numpy=True)
toc = time.time()
print('Test encoding time:', toc-tic)

passage_embeddings = model._do_encode(passages, batch_size=batch_size, instruction=passage_prefix, max_length=max_length, num_workers=32, return_numpy=True)
toc = time.time()
print('Total encoding time:', toc-tic)

# normalize embeddings
# query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
# passage_embeddings = F.normalize(passage_embeddings, p=2, dim=1)
query_embeddings = F.normalize(torch.tensor(query_embeddings), p=2, dim=1)
passage_embeddings = F.normalize(torch.tensor(passage_embeddings), p=2, dim=1)

scores = (query_embeddings @ passage_embeddings.T) * 100
# print(scores.tolist())
# [[87.42692565917969, 0.462837278842926], [0.9652643203735352, 86.0372314453125]]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

encoding:   0%|          | 0/127 [00:01<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Test encoding time: 36.73291730880737


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

encoding:   0%|          | 0/100 [00:01<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Total encoding time: 63.64254975318909


In [15]:
scores

tensor([[78.6956, 81.3270, 79.4460,  ..., 84.1884, 83.9234, 80.1651],
        [79.3121, 82.8288, 81.6716,  ..., 83.2478, 83.0006, 82.4287],
        [80.0940, 78.5115, 76.5741,  ..., 81.8465, 82.3200, 77.8372],
        ...,
        [75.2231, 81.9015, 80.3835,  ..., 79.8307, 79.5554, 83.6244],
        [77.8370, 83.3969, 80.9174,  ..., 82.8269, 82.7837, 82.1167],
        [71.1475, 79.2757, 82.5588,  ..., 78.2288, 77.2570, 79.6065]])

In [21]:
import numpy as np

def get_top_k_similar_users(scores_matrix, query_user_names, corpus_user_names, k):
    top_k_similar_users = {}
    scores_matrix = scores_matrix.detach().cpu().numpy()

    for i, curr_scores in enumerate(scores_matrix):
        # Get the indices of the top k scores
        top_k_indices = np.argsort(curr_scores)[-k:][::-1]
        
        # Map indices to corresponding user names in the corpus_user_names list
        top_k_users = [corpus_user_names[idx] for idx in top_k_indices]
        
        # Map the query user name to the list of top k similar users
        top_k_similar_users[query_user_names[i]] = top_k_users

    return top_k_similar_users

k = 20  # Number of top similar users to retrieve
result = get_top_k_similar_users(scores, test_user_list, train_user_list, k)
print(result)


{'1622': ['92', '195', '125', '135', '233', '221', '17', '133', '234', '58', '19', '162', '112', '124', '142', '157', '197', '215', '201', '96'], '63': ['101', '40', '160', '71', '92', '189', '221', '168', '125', '17', '58', '231', '195', '229', '120', '152', '233', '62', '133', '50'], '697': ['122', '47', '19', '201', '53', '172', '195', '83', '133', '82', '135', '234', '162', '125', '197', '96', '33', '86', '233', '183'], '566': ['120', '152', '223', '177', '100', '35', '205', '50', '103', '199', '73', '109', '92', '149', '42', '34', '43', '40', '76', '185'], '1868': ['135', '195', '92', '125', '112', '221', '58', '233', '17', '234', '162', '24', '96', '133', '183', '101', '142', '44', '124', '215'], '1716': ['162', '135', '105', '86', '198', '41', '172', '44', '125', '134', '70', '126', '13', '83', '19', '178', '77', '82', '209', '54'], '2263': ['151', '210', '155', '189', '108', '231', '74', '236', '141', '140', '216', '27', '20', '65', '64', '87', '190', '164', '208', '84'], '220'